In [ ]:
from collections import defaultdict
import glob
from time import time

import numpy as np
import pandas as pd
import networkx as nx

import sys
sys.path.append('../../algorithms')
from SPOC import SPOC
from generate_SPOC_model import *

import matlab.engine
from tqdm import tqdm_notebook
from IPython.display import clear_output, display

import random
random.seed(42)
np.random.seed(42)

eng = matlab.engine.start_matlab()
_ = eng.addpath('../../algorithms')

In [ ]:
def load_data(adjacency_filepath, community_filepath):
    G=nx.read_edgelist(adjacency_filepath, nodetype=int)
    nodelist = list(G.nodes())
    node2indx = {n: i for i, n in enumerate(nodelist)}
    A = nx.to_scipy_sparse_matrix(G, nodelist=nodelist)
    comms = {}

    with open(community_filepath) as f:
        for line in f:
            args = line.split()
            k = int(args[1]) - 1
            if k not in comms:
                comms[k] = ([], [])
            comms[k][0].append(node2indx[int(args[0])])
            comms[k][1].append(float(args[2]))

    comm_count = len(comms)
    comms_array = np.zeros((A.shape[0], comm_count))
    for i in range(comm_count):
        comms_array[comms[i][0], i] = comms[i][1]
    return nodelist, A, comms_array

In [ ]:
# If you need particular data
# import re
# need_data = ['DBLP2', 'DBLP1']
# need_data = "|".join(need_data)

# data_paths = glob.glob('../../data/coauthorship/*_adjacency.txt')
# if need_data != []:
#     data_paths = filter(lambda x: re.findall(need_data, x) != [], data_paths)
# data_paths
# data_paths = [(adj, adj.replace("adjacency", "community"))for adj in data_paths]
# print("\n".join("{}, {}".format(*data) for data in data_paths))

In [ ]:
## Extract coauthorship.zip
data_paths = glob.glob('../../data/coauthorship/*_adjacency.txt')
data_paths = [(adj, adj.replace("adjacency", "community"))for adj in data_paths]
print("\n".join("{}, {}".format(*data) for data in data_paths))

#### SPOC methods from A matrix

In [ ]:
methods = {
    "SPOC": lambda A, n_clusters: model.fit(A=A, n_clusters=n_clusters, use_cvxpy=True,
                                            use_bootstrap=False, use_ellipsoid=True),
    "GeoNMF": lambda A, n_clusters: eng.GeoNMF(A, n_clusters, 0.25, 0.95, nargout=2),
    "SPOC_bootstrap": lambda A, n_clusters: model.fit(A=A, n_clusters=n_clusters, std_num=3,
                                                      use_bootstrap=True, n_repetitions=10, )
          }

results = []
for method_name in tqdm_notebook(methods):
    print('\t', method_name)
    for data in tqdm_notebook(data_paths):
        print(time())
        nodelist, A, comms = load_data(*data)
        n_clusters = comms.shape[1]
        A_mat = matlab.double(A.todense().tolist())
        print(data, '...', 'clusters:', n_clusters, "nodes: ", A.shape[0])
        time_start = time()
        theta, b = methods[method_name](A_mat, n_clusters)
        time_end = time()
        res = {"relative MSE": find_permutation_Theta(comms, theta)[0], 
               "Mean Spearman coefficient": find_permutation_spearmanr(comms, theta)[0],
               "Method": method_name,
               "Dataset": data[0].split("/")[-1].split("_")[0],
               "cluster #": n_clusters, 
               "node #": A.shape[0],
               "time (sec)": time_end - time_start,
              }
        results.append(res)
        print(res)

In [ ]:
EXP_DATA = pd.DataFrame(results)
EXP_DATA.to_csv('../../results/real_data.csv')